# INF582 AXA Challenge

## Initialisation

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# Load main librairies
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import itertools
import random

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import __version__
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import train_test_split

pd.set_option('display.max_columns', 500)

## Load data

In [35]:
# Load data
submission = pd.read_csv("data/submission.txt", sep='\t')
training_data = pd.read_csv("data/train_2011_2012.csv", sep=';', nrows=10000,
                            na_values=['A Définir', 'A DEFINIR', '9999-12-31 00:00:00.000'],
                            usecols=['DATE','WEEK_END','DAY_WE_DS','TPER_TEAM','ASS_ASSIGNMENT','CSPL_RECEIVED_CALLS']
                            )
training_data.head()

,DATE,WEEK_END,DAY_WE_DS,TPER_TEAM,ASS_ASSIGNMENT,CSPL_RECEIVED_CALLS
0,2011-04-24 01:30:00.000,1,Dimanche,Nuit,Téléphonie,0
1,2011-04-24 01:30:00.000,1,Dimanche,Nuit,Finances PCX,0
2,2011-04-24 01:30:00.000,1,Dimanche,Nuit,Finances PCX,0
3,2011-04-24 01:30:00.000,1,Dimanche,Nuit,Téléphonie,0
4,2011-04-24 01:30:00.000,1,Dimanche,Nuit,Téléphonie,0


In [36]:
meteo2012 = pd.read_csv("data/meteo_2012.csv",nrows=10000,
                    header=None,
                    usecols=[0,1,3,6]
                   )

meteo2012.rename(columns={0: 'DATE', 1: 'DEPT',3:'TEMP',6:'RAIN'}, inplace=True)
meteo2012 = meteo2012[meteo2012[['DEPT']].apply(lambda x: x[0].isdigit(), axis=1)]
meteo2012 = meteo2012[~meteo2012.DEPT.isin(['00'])]

meteo2011 = pd.read_csv("data/meteo_2011.csv",nrows=10000,
                    header=None,
                    usecols=[0,1,3,6]
                   )
meteo2011.rename(columns={0: 'DATE', 1: 'DEPT',3:'TEMP',6:'RAIN'}, inplace=True)
meteo2011 = meteo2011[meteo2011[['DEPT']].apply(lambda x: x[0].isdigit(), axis=1)]
meteo2011 = meteo2011[~meteo2011.DEPT.isin(['00'])]

frames=[meteo2012,meteo2011]

meteo=pd.concat(frames)

def format_date(date):
    return dt.datetime.strptime(date, '%Y-%m-%d %H:%M').strftime('%Y-%m-%d %H:%M:%S.000')

meteo['DATE'] = meteo.DATE.map(format_date)

#Format meteo
for dept in sorted(meteo.DEPT.unique()):
    index = (meteo.DEPT == dept)
    meteo["{}_TEMP".format(dept)] = meteo.TEMP * index
    meteo["{}_RAIN".format(dept)] = meteo.RAIN * index

meteo.drop(["DEPT", "RAIN", "TEMP"], axis=1, inplace=True)
meteo = meteo.groupby("DATE").sum().reset_index()

meteo.head()


,DATE,01_TEMP,01_RAIN,02_TEMP,02_RAIN,03_TEMP,03_RAIN,04_TEMP,04_RAIN,05_TEMP,05_RAIN,06_TEMP,06_RAIN,07_TEMP,07_RAIN,08_TEMP,08_RAIN,09_TEMP,09_RAIN,10_TEMP,10_RAIN,11_TEMP,11_RAIN,12_TEMP,12_RAIN,13_TEMP,13_RAIN,14_TEMP,14_RAIN,15_TEMP,15_RAIN,16_TEMP,16_RAIN,17_TEMP,17_RAIN,18_TEMP,18_RAIN,19_TEMP,19_RAIN,20_TEMP,20_RAIN,21_TEMP,21_RAIN,22_TEMP,22_RAIN,23_TEMP,23_RAIN,24_TEMP,24_RAIN,25_TEMP,25_RAIN,26_TEMP,26_RAIN,27_TEMP,27_RAIN,28_TEMP,28_RAIN,29_TEMP,29_RAIN,30_TEMP,30_RAIN,31_TEMP,31_RAIN,32_TEMP,32_RAIN,33_TEMP,33_RAIN,34_TEMP,34_RAIN,35_TEMP,35_RAIN,36_TEMP,36_RAIN,37_TEMP,37_RAIN,38_TEMP,38_RAIN,39_TEMP,39_RAIN,40_TEMP,40_RAIN,41_TEMP,41_RAIN,42_TEMP,42_RAIN,43_TEMP,43_RAIN,44_TEMP,44_RAIN,45_TEMP,45_RAIN,46_TEMP,46_RAIN,47_TEMP,47_RAIN,48_TEMP,48_RAIN,49_TEMP,49_RAIN,50_TEMP,50_RAIN,51_TEMP,51_RAIN,52_TEMP,52_RAIN,53_TEMP,53_RAIN,54_TEMP,54_RAIN,56_TEMP,56_RAIN,57_TEMP,57_RAIN,58_TEMP,58_RAIN,59_TEMP,59_RAIN,60_TEMP,60_RAIN,61_TEMP,61_RAIN,62_TEMP,62_RAIN,63_TEMP,63_RAIN,64_TEMP,64_RAIN,65_TEMP,65_RAIN,66_TEMP,66_RAIN,67_TEMP,67_RAIN,68_TEMP,68_RAIN,69_TEMP,69_RAIN,70_TEMP,70_RAIN,71_TEMP,71_RAIN,72_TEMP,72_RAIN,73_TEMP,73_RAIN,74_TEMP,74_RAIN,75_TEMP,75_RAIN,76_TEMP,76_RAIN,77_TEMP,77_RAIN,78_TEMP,78_RAIN,79_TEMP,79_RAIN,80_TEMP,80_RAIN,81_TEMP,81_RAIN,82_TEMP,82_RAIN,83_TEMP,83_RAIN,84_TEMP,84_RAIN,85_TEMP,85_RAIN,86_TEMP,86_RAIN,87_TEMP,87_RAIN,88_TEMP,88_RAIN,89_TEMP,89_RAIN,90_TEMP,90_RAIN,91_TEMP,91_RAIN,93_TEMP,93_RAIN,94_TEMP,94_RAIN,95_TEMP,95_RAIN
0,2011-01-01 00:00:00.000,0.6,0,-1.3,0,3.5,0,-5.3,0.0,-8.1,0,15.4,0,6.6,0,-0.1,0,2.5,0,-0.6,0,27.8,0,12.6,0,50.7,0,-3.6,0,16.2,0,8.6,0,31.1,0,4.0,0,13.4,0,0,0,0.1,0,8.7,0,6.4,0,1.8,0,0.4,0,21.3,0,-1.2,0,-1.1,0,34.8,0,30.1,0,21.9,0,8.9,0,26.1,0,42.5,0,4.2,0,1.7,0,0.5,0,8.8,0,0.8,0,18.1,0,9.000000e-01,0,6.5,0,3.6,0,7.8,0,-0.6,0,9.7,0,3.4,0,10.8,0,2.5,0,10.4,0,-1.8,0,-2.8,0,0.3,0,-3.2,0,9.1,0,-5.6,0,2.6,0,9.8,0.0,-1.4,0,-0.8,0,8.2,0,4.1,0,30.3,0,2.3,0,20.4,0.0,-0.3,0,-1.9,0,4.2,0,0.6,0,3.0,0,1.3,0,-1.4,0,-2.5,0,0.1,0,-3.3,0,-0.3,0,-2.2,0,7.8,0,-0.6,0,8.9,0,7.6,0,77.9,0.0,23.9,0,13.4,0,5.3,0,6.4,0,0,0,0.1,0,-1.0,0,-0.6,0,-0.7,0,-0.5,0,-1.9,0
1,2011-01-01 01:00:00.000,-0.6,0,-1.2,0,3.4,0,-5.8,0.0,-9.6,0,14.8,0,6.5,0,0.0,0,2.5,0,-0.6,0,27.9,0,12.6,0,51.0,0,-4.1,0,15.9,0,8.6,0,29.3,0,3.5,0,13.2,0,0,0,0.0,0,8.3,0,6.2,0,1.9,0,0.0,0,20.4,0,-1.4,0,-1.3,0,32.9,0,29.6,0,19.9,0,8.5,0,23.4,0,40.9,0,3.5,0,1.6,0,0.2,0,6.5,0,0.7,0,17.2,0,6.000000e-01,0,3.7,0,3.3,0,4.4,0,-1.0,0,9.6,0,2.2,0,10.3,0,2.1,0,9.8,0,-1.7,0,-3.0,0,0.2,0,-3.3,0,8.7,0,-4.7,0,2.5,0,9.8,0.0,-1.4,0,-1.0,0,7.5,0,3.9,0,28.9,0,2.6,0,20.4,0.0,-0.4,0,-1.8,0,3.4,0,0.5,0,2.4,0,1.2,0,-1.5,0,-2.0,0,0.2,0,-3.8,0,-0.3,0,-2.4,0,7.1,0,-0.7,0,8.8,0,6.1,0,77.2,0.0,24.1,0,12.4,0,4.9,0,5.9,0,0,0,-0.1,0,-1.0,0,0.0,0,-1.1,0,-0.5,0,-1.8,0
2,2011-01-01 02:00:00.000,-0.7,0,-1.0,0,3.3,0,-4.3,0.0,-10.0,0,14.4,0,6.3,0,0.0,0,2.5,0,-0.7,0,27.0,0,12.3,0,50.8,0,-4.6,0,15.7,0,8.6,0,27.8,0,2.6,0,12.4,0,0,0,-0.1,0,8.1,0,6.2,0,3.8,0,-0.2,0,19.3,0,-1.5,0,-1.7,0,31.4,0,29.2,0,17.9,0,6.6,0,21.6,0,39.4,0,3.1,0,1.7,0,0.1,0,4.1,0,0.7,0,16.7,0,5.000000e-01,0,3.2,0,3.0,0,4.0,0,-0.5,0,9.2,0,1.6,0,9.8,0,1.0,0,9.7,0,-1.6,0,-3.2,0,-0.1,0,-2.8,0,8.1,0,-4.4,0,2.5,0,9.8,0.0,-1.3,0,-1.2,0,7.1,0,3.8,0,28.0,0,2.6,0,20.0,0.0,-0.5,0,-1.8,0,2.8,0,0.3,0,1.9,0,0.6,0,-1.3,0,-2.2,0,0.2,0,-4.4,0,-0.5,0,-2.5,0,6.8,0,-0.5,0,8.7,0,6.3,0,76.3,0.2,23.9,0,7.3,0,4.2,0,6.0,0,0,0,-0.3,0,-1.0,0,-0.6,0,-1.0,0,-0.6,0,-1.8,0
3,2011-01-01 03:00:00.000,-0.8,0,-0.8,0,3.3,0,-1.3,0.2,-8.2,0,13.6,0,6.0,0,0.1,0,2.5,0,-0.8,0,26.5,0,12.2,0,49.2,0,-4.7,0,15.4,0,8.4,0,25.6,0,2.5,0,11.5,0,0,0,-0.1,0,8.7,0,6.0,0,3.5,0,-0.4,0,18.4,0,-1.6,0,-1.7,0,29.7,0,28.5,0,17.6,0,4.5,0,22.8,0,37.0,0,2.2,0,1.7,0,-0.1,0,3.4,0,0.8,0,17.5,0,1.000000e-01,0,4.9,0,2.9,0,3.4,0,-0.8,0,8.7,0,0.7,0,9.3,0,0.5,0,8.3,0,-1.6,0,-3.4,0,-0.2,0,-2.6,0,7.6,0,-4.2,0,2.6,0,9.9,0.2,-0.9,0,-1.2,0,6.8,0,3.7,0,25.2,0,2.4,0,19.7,0.4,-0.5,0,-1.7,0,2.1,0,0.2,0,1.7,0,0.4,0,-1.4,0,-3.3,0,0.2,0,-4.1,0,-0.6,0,-2.6,0,5.6,0,0.6,0,8.7,0,7.0,0,

## Clean data

In [37]:
# Remove non usefull rows
training_data = training_data[training_data.ASS_ASSIGNMENT.isin(submission.ASS_ASSIGNMENT.unique())]

In [38]:
# Aggregate the calls
training_data = training_data.groupby([col for col in training_data.columns if not col == 'CSPL_RECEIVED_CALLS']).sum().reset_index()

In [39]:
#Extract the time slot from date
def get_time(date):
    time = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').time()
    return time.hour * 3600 + time.minute * 60 + time.second

def get_month(date):
    month = dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').month
    return month

#Join weather
training_data = pd.merge(training_data, meteo, how='left', on=['DATE'])

#Deal with NAs. Would be better to replace with average
training_data.fillna(0,inplace = True)

#Assign a number to the day of the week
day_to_num_dict = {j:i for i,j in enumerate(['Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi','Dimanche'])}

training_data['TIME'] = training_data.DATE.map(get_time)
training_data['MONTH'] = training_data.DATE.map(get_month)
training_data['WEEK_DAY'] = training_data.DAY_WE_DS.map(day_to_num_dict)
training_data['NIGHT'] = (training_data.TPER_TEAM == "Nuit") * 1

# Remove obsolete columns
training_data = training_data[[col for col in training_data.columns if not col in ['DATE','DAY_WE_DS','TPER_TEAM']]]

In [40]:
#Convert the different ASS_ASSIGNMENTs to booleans
for value in submission.ASS_ASSIGNMENT.unique():
    training_data["ASS_ASSIGNMENT_"+value] = (training_data.ASS_ASSIGNMENT == value) * 1
    
# Remove obsolete column
training_data = training_data[[col for col in training_data.columns if not col == 'ASS_ASSIGNMENT']]

In [41]:
training_data.head()

,WEEK_END,CSPL_RECEIVED_CALLS,01_TEMP,01_RAIN,02_TEMP,02_RAIN,03_TEMP,03_RAIN,04_TEMP,04_RAIN,05_TEMP,05_RAIN,06_TEMP,06_RAIN,07_TEMP,07_RAIN,08_TEMP,08_RAIN,09_TEMP,09_RAIN,10_TEMP,10_RAIN,11_TEMP,11_RAIN,12_TEMP,12_RAIN,13_TEMP,13_RAIN,14_TEMP,14_RAIN,15_TEMP,15_RAIN,16_TEMP,16_RAIN,17_TEMP,17_RAIN,18_TEMP,18_RAIN,19_TEMP,19_RAIN,20_TEMP,20_RAIN,21_TEMP,21_RAIN,22_TEMP,22_RAIN,23_TEMP,23_RAIN,24_TEMP,24_RAIN,25_TEMP,25_RAIN,26_TEMP,26_RAIN,27_TEMP,27_RAIN,28_TEMP,28_RAIN,29_TEMP,29_RAIN,30_TEMP,30_RAIN,31_TEMP,31_RAIN,32_TEMP,32_RAIN,33_TEMP,33_RAIN,34_TEMP,34_RAIN,35_TEMP,35_RAIN,36_TEMP,36_RAIN,37_TEMP,37_RAIN,38_TEMP,38_RAIN,39_TEMP,39_RAIN,40_TEMP,40_RAIN,41_TEMP,41_RAIN,42_TEMP,42_RAIN,43_TEMP,43_RAIN,44_TEMP,44_RAIN,45_TEMP,45_RAIN,46_TEMP,46_RAIN,47_TEMP,47_RAIN,48_TEMP,48_RAIN,49_TEMP,49_RAIN,50_TEMP,50_RAIN,51_TEMP,51_RAIN,52_TEMP,52_RAIN,53_TEMP,53_RAIN,54_TEMP,54_RAIN,56_TEMP,56_RAIN,57_TEMP,57_RAIN,58_TEMP,58_RAIN,59_TEMP,59_RAIN,60_TEMP,60_RAIN,61_TEMP,61_RAIN,62_TEMP,62_RAIN,63_TEMP,63_RAIN,64_TEMP,64_RAIN,65_TEMP,65_RAIN,66_TEMP,66_RAIN,67_TEMP,67_RAIN,68_TEMP,68_RAIN,69_TEMP,69_RAIN,70_TEMP,70_RAIN,71_TEMP,71_RAIN,72_TEMP,72_RAIN,73_TEMP,73_RAIN,74_TEMP,74_RAIN,75_TEMP,75_RAIN,76_TEMP,76_RAIN,77_TEMP,77_RAIN,78_TEMP,78_RAIN,79_TEMP,79_RAIN,80_TEMP,80_RAIN,81_TEMP,81_RAIN,82_TEMP,82_RAIN,83_TEMP,83_RAIN,84_TEMP,84_RAIN,85_TEMP,85_RAIN,86_TEMP,86_RAIN,87_TEMP,87_RAIN,88_TEMP,88_RAIN,89_TEMP,89_RAIN,90_TEMP,90_RAIN,91_TEMP,91_RAIN,93_TEMP,93_RAIN,94_TEMP,94_RAIN,95_TEMP,95_RAIN,TIME,MONTH,WEEK_DAY,NIGHT,ASS_ASSIGNMENT_CAT,ASS_ASSIGNMENT_Téléphonie,ASS_ASSIGNMENT_Tech. Inter,ASS_ASSIGNMENT_Tech. Axa,ASS_ASSIGNMENT_Services,ASS_ASSIGNMENT_Regulation Medicale,ASS_ASSIGNMENT_RENAULT,ASS_ASSIGNMENT_Nuit,ASS_ASSIGNMENT_SAP,ASS_ASSIGNMENT_Japon,ASS_ASSIGNMENT_Gestion Renault,ASS_ASSIGNMENT_Gestion Amex,ASS_ASSIGNMENT_Gestion - Accueil Telephonique,ASS_ASSIGNMENT_Gestion,ASS_ASSIGNMENT_Domicile,ASS_ASSIGNMENT_Crises,ASS_ASSIGNMENT_Médical,ASS_ASSIGNMENT_Tech. Total,ASS_ASSIGNMENT_Mécanicien,ASS_ASSIGNMENT_Gestion Relation Clienteles,ASS_ASSIGNMENT_Manager,ASS_ASSIGNMENT_Gestion Clients,ASS_ASSIGNMENT_Gestion DZ,ASS_ASSIGNMENT_RTC,ASS_ASSIGNMENT_CMS,ASS_ASSIGNMENT_Prestataires,ASS_ASSIGNMENT_Gestion Assurances
0,1,0,0.6,0,-1.3,0,3.5,0,-5.3,0,-8.1,0,15.4,0,6.6,0,-0.1,0,2.5,0,-0.6,0,27.8,0,12.6,0,50.7,0,-3.6,0,16.2,0,8.6,0,31.1,0,4,0,13.4,0,0,0,0.1,0,8.7,0,6.4,0,1.8,0,0.4,0,21.3,0,-1.2,0,-1.1,0,34.8,0,30.1,0,21.9,0,8.9,0,26.1,0,42.5,0,4.2,0,1.7,0,0.5,0,8.8,0,0.8,0,18.1,0,0.9,0,6.5,0,3.6,0,7.8,0,-0.6,0,9.7,0,3.4,0,10.8,0,2.5,0,10.4,0,-1.8,0,-2.8,0,0.3,0,-3.2,0,9.1,0,-5.6,0,2.6,0,9.8,0,-1.4,0,-0.8,0,8.2,0,4.1,0,30.3,0,2.3,0,20.4,0,-0.3,0,-1.9,0,4.2,0,0.6,0,3,0,1.3,0,-1.4,0,-2.5,0,0.1,0,-3.3,0,-0.3,0,-2.2,0,7.8,0,-0.6,0,8.9,0,7.6,0,77.9,0,23.9,0,13.4,0,5.3,0,6.4,0,0,0,0.1,0,-1,0,-0.6,0,-0.7,0,-0.5,0,-1.9,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0.6,0,-1.3,0,3.5,0,-5.3,0,-8.1,0,15.4,0,6.6,0,-0.1,0,2.5,0,-0.6,0,27.8,0,12.6,0,50.7,0,-3.6,0,16.2,0,8.6,0,31.1,0,4,0,13.4,0,0,0,0.1,0,8.7,0,6.4,0,1.8,0,0.4,0,21.3,0,-1.2,0,-1.1,0,34.8,0,30.1,0,21.9,0,8.9,0,26.1,0,42.5,0,4.2,0,1.7,0,0.5,0,8.8,0,0.8,0,18.1,0,0.9,0,6.5,0,3.6,0,7.8,0,-0.6,0,9.7,0,3.4,0,10.8,0,2.5,0,10.4,0,-1.8,0,-2.8,0,0.3,0,-3.2,0,9.1,0,-5.6,0,2.6,0,9.8,0,-1.4,0,-0.8,0,8.2,0,4.1,0,30.3,0,2.3,0,20.4,0,-0.3,0,-1.9,0,4.2,0,0.6,0,3,0,1.3,0,-1.4,0,-2.5,0,0.1,0,-3.3,0,-0.3,0,-2.2,0,7.8,0,-0.6,0,8.9,0,7.6,0,77.9,0,23.9,0,13.4,0,5.3,0,6.4,0,0,0,0.1,0,-1,0,-0.6,0,-0.7,0,-0.5,0,-1.9,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0.6,0,-1.3,0,3.5,0,-5.3,0,-8.1,0,15.4,0,6.6,0,-0.1,0,2.5,0,-0.6,0,27.8,0,12.6,0,50.7,0,-3.6,0,16.2,0,8.6,0,31.1,0,4,0,13.4,0,0,0,0.1,0,8.7,0,6.4,0,1.8,0,0.4,0,21.3,0,-1.2,0,-1.1,0,34.8,0,30.1,0,21.9,0,8.9,0,26.1,0,42.5,0,4.2,0,1.7,0,0.5,0,8.8,0,0.8,0,18.1,0,0.9,0,6.5,0,3.6,0,7.8,0,-0.6,0,9.7,0,3.4,0,10.8,0,2.5,0,10.4,0,-1.8,0,-2.8,0,0.3,0,-3.2,0,9.1,0,-5.6,0,2.6,0,9.8,0,-1.4,0,-0.8,0,8.2,0,4.1,0,30.3,0,2.

## View main statistics

In [42]:
training_data.describe()

,WEEK_END,CSPL_RECEIVED_CALLS,01_TEMP,01_RAIN,02_TEMP,02_RAIN,03_TEMP,03_RAIN,04_TEMP,04_RAIN,05_TEMP,05_RAIN,06_TEMP,06_RAIN,07_TEMP,07_RAIN,08_TEMP,08_RAIN,09_TEMP,09_RAIN,10_TEMP,10_RAIN,11_TEMP,11_RAIN,12_TEMP,12_RAIN,13_TEMP,13_RAIN,14_TEMP,14_RAIN,15_TEMP,15_RAIN,16_TEMP,16_RAIN,17_TEMP,17_RAIN,18_TEMP,18_RAIN,19_TEMP,19_RAIN,20_TEMP,20_RAIN,21_TEMP,21_RAIN,22_TEMP,22_RAIN,23_TEMP,23_RAIN,24_TEMP,24_RAIN,25_TEMP,25_RAIN,26_TEMP,26_RAIN,27_TEMP,27_RAIN,28_TEMP,28_RAIN,29_TEMP,29_RAIN,30_TEMP,30_RAIN,31_TEMP,31_RAIN,32_TEMP,32_RAIN,33_TEMP,33_RAIN,34_TEMP,34_RAIN,35_TEMP,35_RAIN,36_TEMP,36_RAIN,37_TEMP,37_RAIN,38_TEMP,38_RAIN,39_TEMP,39_RAIN,40_TEMP,40_RAIN,41_TEMP,41_RAIN,42_TEMP,42_RAIN,43_TEMP,43_RAIN,44_TEMP,44_RAIN,45_TEMP,45_RAIN,46_TEMP,46_RAIN,47_TEMP,47_RAIN,48_TEMP,48_RAIN,49_TEMP,49_RAIN,50_TEMP,50_RAIN,51_TEMP,51_RAIN,52_TEMP,52_RAIN,53_TEMP,53_RAIN,54_TEMP,54_RAIN,56_TEMP,56_RAIN,57_TEMP,57_RAIN,58_TEMP,58_RAIN,59_TEMP,59_RAIN,60_TEMP,60_RAIN,61_TEMP,61_RAIN,62_TEMP,62_RAIN,63_TEMP,63_RAIN,64_TEMP,64_RAIN,65_TEMP,65_RAIN,66_TEMP,66_RAIN,67_TEMP,67_RAIN,68_TEMP,68_RAIN,69_TEMP,69_RAIN,70_TEMP,70_RAIN,71_TEMP,71_RAIN,72_TEMP,72_RAIN,73_TEMP,73_RAIN,74_TEMP,74_RAIN,75_TEMP,75_RAIN,76_TEMP,76_RAIN,77_TEMP,77_RAIN,78_TEMP,78_RAIN,79_TEMP,79_RAIN,80_TEMP,80_RAIN,81_TEMP,81_RAIN,82_TEMP,82_RAIN,83_TEMP,83_RAIN,84_TEMP,84_RAIN,85_TEMP,85_RAIN,86_TEMP,86_RAIN,87_TEMP,87_RAIN,88_TEMP,88_RAIN,89_TEMP,89_RAIN,90_TEMP,90_RAIN,91_TEMP,91_RAIN,93_TEMP,93_RAIN,94_TEMP,94_RAIN,95_TEMP,95_RAIN,TIME,MONTH,WEEK_DAY,NIGHT,ASS_ASSIGNMENT_CAT,ASS_ASSIGNMENT_Téléphonie,ASS_ASSIGNMENT_Tech. Inter,ASS_ASSIGNMENT_Tech. Axa,ASS_ASSIGNMENT_Services,ASS_ASSIGNMENT_Regulation Medicale,ASS_ASSIGNMENT_RENAULT,ASS_ASSIGNMENT_Nuit,ASS_ASSIGNMENT_SAP,ASS_ASSIGNMENT_Japon,ASS_ASSIGNMENT_Gestion Renault,ASS_ASSIGNMENT_Gestion Amex,ASS_ASSIGNMENT_Gestion - Accueil Telephonique,ASS_ASSIGNMENT_Gestion,ASS_ASSIGNMENT_Domicile,ASS_ASSIGNMENT_Crises,ASS_ASSIGNMENT_Médical,ASS_ASSIGNMENT_Tech. Total,ASS_ASSIGNMENT_Mécanicien,ASS_ASSIGNMENT_Gestion Relation Clienteles,ASS_ASSIGNMENT_Manager,ASS_ASSIGNMENT_Gestion Clients,ASS_ASSIGNMENT_Gestion DZ,ASS_ASSIGNMENT_RTC,ASS_ASSIGNMENT_CMS,ASS_ASSIGNMENT_Prestataires,ASS_ASSIGNMENT_Gestion Assurances
count,1584.000000,1584.000000,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584,1584.000000,1584.000000,1584.000000,1584,1584.000000,1584.000000,1584.000000,1584.000000,1584.000000,1584.000000,1584.000000,1584.000000,1584.00000

##  A simple predictor

Lets try building a tree-based boosting predictor with very few attributes just to see how it goes.
This predictor will only predict the number of calls received during a given time stamp (e.g. in a 30 minutes slot) and day of week.

Note that the date is not relevant for regression, but we can extract some relevant information from it: day of the week, time slot, and if it is a week-end or not.

Also, for some reason the data for a given ASS_ASSIGNMENT and DATE is sometimes split, so we have to aggregate it.

In [43]:
output_cols = ['CSPL_RECEIVED_CALLS']
input_cols = [col for col in training_data.columns if not col in output_cols]

In [44]:
#Now create the gradient boosting regressor

X_train, X_test, y_train, y_test = train_test_split(training_data[input_cols],  training_data[output_cols].values.ravel())

est = GradientBoostingRegressor()

tuned_parameters = {'loss' : ['ls', 'lad', 'huber', 'quantile'],'n_estimators':[30],'learning_rate': [0.3,0.5,0.7], 'subsample': [1.0],
                     'min_samples_split':[1,3],'min_samples_leaf':[1,2],
                     'max_depth':[3,5],'max_features':['auto']
                    }
                     

clf = RandomizedSearchCV(est, tuned_parameters, cv=5,n_jobs=-1,n_iter=5,verbose=1)

clf.fit(X_train, y_train)

est_temoin=GradientBoostingRegressor()
est_temoin.fit(X_train,y_train)

print(clf.best_params_)
best_est=clf.best_estimator_

#Plot CV error (this is squared loss, which will be used to evaluate our performance in the leaderboard)

#Sur 10.000 lignes best_estimator est souvent moins bon que le temoin.. Par contre la différence est nette
# quand on utilise toutes les données.
#The higher the score the better.
print("Best estimator : %.4f" %best_est.score(X_test,y_test))
print("Temoin : %.4f" %est_temoin.score(X_test,y_test))



Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'learning_rate': 0.5, 'n_estimators': 30, 'max_depth': 3, 'loss': 'lad', 'subsample': 1.0, 'min_samples_leaf': 1, 'max_features': 'auto', 'min_samples_split': 1}
Best estimator : 0.8880
Temoin : 0.8573


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    2.6s finished


# Prediction and submission

In [45]:
test_data = submission.copy()

#Join weather
test_data = pd.merge(test_data, meteo, how='left', on=['DATE'])

#Deal with NAs. Would be better to replace with average
test_data.fillna(0,inplace = True)

def get_weekday(date):
    return dt.datetime.strptime(date,'%Y-%m-%d %H:%M:%S.000').weekday()

test_data['TIME'] = test_data.DATE.map(get_time)
test_data['MONTH'] = test_data.DATE.map(get_month)
test_data['WEEK_DAY'] = test_data.DATE.map(get_weekday)
test_data['NIGHT'] = (np.logical_or(test_data.TIME >= (23*3600 + 30*60),
                                    test_data.TIME <  (7*3600  + 30*60))) * 1
test_data['WEEK_END'] = test_data.WEEK_DAY.isin([5, 6]) * 1

# Convert the different ASS_ASSIGNMENTs to booleans
for value in submission.ASS_ASSIGNMENT.unique():
    test_data["ASS_ASSIGNMENT_"+value] = (test_data.ASS_ASSIGNMENT == value) * 1

test_data = test_data[input_cols]

In [46]:
submission.prediction = best_est.predict(test_data)
submission.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-01-03 00:00:00.000,CAT,0
1,2012-01-03 00:00:00.000,Téléphonie,1
2,2012-01-03 00:00:00.000,Tech. Inter,0
3,2012-01-03 00:00:00.000,Tech. Axa,0
4,2012-01-03 00:00:00.000,Services,0


In [47]:
# Write prediction to csv
submission.to_csv("data/output.txt", sep='\t', index=False)